<a href="https://colab.research.google.com/github/kshakib22/Newspaper-Template-Matching/blob/main/CSV_cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
%%capture
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
import pandas as pd
import pandas as pd
from datetime import datetime, timedelta
import re
import numpy as np
from dateutil.parser import parse
import pandas as pd

Mounted at /content/drive


In [65]:
# Import the csv file to be cleaned
csv_file_path = '/content/drive/MyDrive/Summer 2023 project/shipping_csv/PE0001107/PE0001107_0008_9.csv'

# Merge dates with months, replace "" with previous date &/ month

Current code should replace `""` with the previous cell value. This should apply to `,,` and `NaN` as well but should be checked with `print(df)` command. This should be experimented with different formats to see if the specific code applied to that type of error.

In [66]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Get the column names and their positions
columns = df.columns.tolist()
print(columns)
positions = {name: i for i, name in enumerate(columns)}

# Check if the "Date" column exists and its position is followed by "Unnamed: 1" column
# If the date number is in Unnamed column, then merge to Date column
if 'Date' in columns and 'Unnamed: 1' in columns and positions['Date'] + 1 == positions['Unnamed: 1']:
    print('merging...')
    # Merge "Unnamed: 1" column with "Date" column
    df['Unnamed: 1'] = df['Unnamed: 1'].astype(str)
    df['Date'] = df['Date'] + df['Unnamed: 1']

    # Drop the "Unnamed: 1" column
    df = df.drop('Unnamed: 1', axis=1)

# Remove all spaces in a column named 'Date' in the DataFrame 'df'
df['Date'] = df['Date'].str.replace(' ', '')
# Replace empty cells or cells containing only a single space with NaN
df['Date'] = df['Date'].replace(r'^\s*$', np.nan, regex=True)
# Replace cells containing only "" or " with NaN
df['Date'] = df['Date'].replace('""', np.nan)
df['Date'] = df['Date'].replace('"', np.nan)
df['Date'] = df['Date'].replace('', np.nan)
df['Date'] = df['Date'].replace('-', 1)


# Iterate over the DataFrame rows
previous_value = None
for i in range(len(df)):
    if pd.isna(df.loc[i, 'Date']):
        df.loc[i, 'Date'] = df.loc[i-1, 'Date']
    else:
        df.loc[i, 'Date'] = df.loc[i, 'Date'].replace('"', '')
        if df.loc[i, 'Date'].isdigit():
            text_match = re.search(r'([A-Za-z]+)', previous_value)
            if text_match:
                previous_value = text_match.group(0)
            df.loc[i, 'Date'] = previous_value + df.loc[i, 'Date']
        else:
            previous_value = df.loc[i, 'Date']

# Replace 'recember' with 'December' in the 'Date' column
# df['Date'] = df['Date'].str.replace('recember', 'December')


# Rename the columns by shifting their positions because of printing error
# df = df.rename(columns={'Consignees': 'Tonnage', 'Cargo': 'Consignees', 'Tonnage': 'Cargo'})


# Display the updated dataframe
print(df)



['Date', 'Unnamed: 1', 'Port', 'Name', 'Tonnage', 'Consignees', 'Cargo']
merging...
         Date         Port           Name  Tonnage        Consignees  \
0   February2       Coloso         Morven     1997         Gilk & Co   
1   February5        Junin      Rauma SS.     1951      L. J. Garrat   
2  February14  Antofagasta      Glenelvan     1756       Gibbs & Co.   
3  February16       Taltal            Dee     1115   Inglis L. & Co.   
4  February17   Valparaiso     Routenburn     1933       J. A. Brown   
5  February20           do   Celtic Queen     1738  Geo. C. Brownell   
6  February20     Coquimbo   John Lockett      779               NaN   
7  February21   Valparaiso       Valdivia      877   Edmondson & Co.   
8  February21      Pisagua  Bay of Biscay     1829               NaN   
9  February21      Iquique       Glenfyne      913               NaN   

       Cargo  
0       Coal  
1    Ballast  
2       Coal  
3       Coal  
4       Coal  
5  C. & Coal  
6    General  
7  

# Convert to date and add year

In [67]:
# Convert the Date column to datetime format if column type is string
if df['Date'].dtype == 'object':
    df['Date'] = df['Date'].apply(lambda x: parse(x + ' 1904', fuzzy=True) if isinstance(x, str) else x)


# Deal with empty values; duplicating Port, Consignees names appearing with do or "

Empty values/NaN in Consignees are replaced with string empty. Any following cells with 'do' is replaced with the previous cell value. Trailing periods are also removed.

In [68]:
# Replace 'do' with the previous cell value in 'Port' column
df['Port'] = df['Port'].replace('do', method='pad')

# Replace 'do' with the previous cell value in 'Consignees' column
df['Consignees'] = df['Consignees'].replace('do', method='pad')

# Replace NaN with 'empty' in 'Port' column
df['Port'] = df['Port'].fillna('empty')

# Remove trailing dots (...) from the Name and Consignees column
df['Name'] = df['Name'].str.rstrip('.')
df['Consignees'] = df['Consignees'].str.rstrip('.')


# Replace NaN with 'empty' in 'Consignees' column
df['Consignees'] = df['Consignees'].fillna('empty')


# Display the updated dataframe
print(df)

        Date         Port           Name  Tonnage        Consignees      Cargo
0 1904-02-02       Coloso         Morven     1997         Gilk & Co       Coal
1 1904-02-05        Junin       Rauma SS     1951      L. J. Garrat    Ballast
2 1904-02-14  Antofagasta      Glenelvan     1756        Gibbs & Co       Coal
3 1904-02-16       Taltal            Dee     1115    Inglis L. & Co       Coal
4 1904-02-17   Valparaiso     Routenburn     1933       J. A. Brown       Coal
5 1904-02-20   Valparaiso   Celtic Queen     1738  Geo. C. Brownell  C. & Coal
6 1904-02-20     Coquimbo   John Lockett      779             empty    General
7 1904-02-21   Valparaiso       Valdivia      877    Edmondson & Co    General
8 1904-02-21      Pisagua  Bay of Biscay     1829             empty       Coal
9 1904-02-21      Iquique       Glenfyne      913             empty    Ballast


# Save output

Cleaned dataframe is saved to path `output` as .csv file

In [69]:
output = '/content/drive/MyDrive/Summer 2023 project/shipping_csv_cleaned/PE0001107/'

# Extract the filename without the extension
filename = os.path.splitext(os.path.basename(csv_file_path))[0]

# Construct the output CSV file path
output_path = os.path.join(output, filename + '.csv')

# Save the DataFrame as a CSV file
df.to_csv(output_path, index=False)
print('File saved at path {}'.format(output_path))

File saved at path /content/drive/MyDrive/Summer 2023 project/shipping_csv_cleaned/PE0001107/PE0001107_0008_9.csv
